<p><font size="7" color='grey'> <b>
Anwendung Generativer KI
</b></font> </br></p>

<p><font size="6" color='grey'> <b>
OutputParser
</b></font> </br></p>


---

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>  Colab-Umfeld</font> </br></p>
# Installierte Python Version
import sys
print(f"Python Version: ",sys.version)
# Installierte LangChain Bibliotheken
print()
print("Installierte LangChain Bibliotheken:")
!pip list | grep '^langchain'
# Unterdrückt die "DeprecationWarning" von LangChain für die Memory-Funktionden
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="langsmith.client")
warnings.filterwarnings("ignore", category=DeprecationWarning, module="langchain")

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>  SetUp API-Keys (setup_api_keys)</font> </br></p>
def setup_api_keys():
    """Konfiguriert alle benötigten API-Keys aus Google Colab userdata"""
    from google.colab import userdata
    import os
    from os import environ

    # Dictionary der benötigten API-Keys
    keys = {
        'OPENAI_API_KEY': 'OPENAI_API_KEY',
        'HF_TOKEN': 'HF_TOKEN',
        # Weitere Keys bei Bedarf
    }

    # Keys in Umgebungsvariablen setzen
    for env_var, key_name in keys.items():
        environ[env_var] = userdata.get(key_name)

    return {k: environ[k] for k in keys.keys()}

# Verwendung
all_keys = setup_api_keys()
# Bei Bedarf einzelne Keys direkt zugreifen
# WEATHER_API_KEY = all_keys['WEATHER_API_KEY']

# **1 | OutputParser**
---

LangChain stellt eine Vielzahl von Ausgabeparsern bereit, die speziell darauf ausgelegt sind, Informationen aus den Ergebnissen großer Sprachmodelle (LLMs) effizient zu extrahieren und zu strukturieren. Diese Parser sind essenzielle Bestandteile der LangChain-Architektur und häufig zentrale Elemente sogenannter LangChain-Ketten. Solche Ketten bestehen aus konfigurierbaren Abfolgen von Operationen, die Modellausgaben verarbeiten und für weiterführende Anwendungen nutzbar machen.

Um die Erstellung und Verwaltung dieser Ketten zu vereinfachen, hat LangChain die LangChain Expression Language (LCEL) entwickelt. Im Folgenden wird erläutert, wie LCEL die erweiterte Konstruktion und Ausführung von LangChain-Ketten unterstützt. Anschließend wird untersucht, wie verschiedene Ausgabeparser innerhalb dieser Ketten eingesetzt werden können, um LLM-Ausgaben gezielt zu analysieren und in verwertbare Informationen zu überführen.

**Warum sind OutputParser so wichtig?**
+ LLMs geben standardmäßig unstrukturierte Texte zurück.
+ OutputParser sind nötig, um das LLM-Output strukturiert weiterzuverarbeiten.
+ Besonders bei komplexen Anwendungen (z. B. Ketten mit mehreren Modellen, Agenten oder RAG-Systemen) müssen die Antworten klar definiert sein.

# **2 | Strukturierter Parser**
---


Der **Structured Output Parser** in LangChain ermöglicht die strukturierte Verarbeitung der Ausgaben großer Sprachmodelle (LLMs). Dies ist besonders hilfreich, wenn Informationen aus mehreren Feldern extrahiert und kategorisiert werden müssen. Durch die Segmentierung der Modellausgabe in definierte Bereiche wird die Interpretation und Handhabung der Daten vereinfacht.

Während der **Pydantic/JSON-Parser** eine leistungsfähigere Lösung für komplexe Datenstrukturen darstellt, eignet sich der **Structured Output Parser** besonders für Umgebungen mit begrenzten Rechenressourcen oder für Modelle mit geringer Leistungsfähigkeit. Er bietet eine einfache und effiziente Möglichkeit, die Ausgabe zu strukturieren, ohne das System zu stark zu belasten.

Der erste Schritt bei der Nutzung dieses Parsers besteht in der Erstellung eines **ResponseSchemas**, das definiert, welche Werte extrahiert werden sollen. Jeder Wert wird dabei detailliert beschrieben. Anschließend wird der **StructuredOutputParser** aus einer Liste dieser Schemata erstellt.

In [ ]:
# Abschnitt 0: Installation und API-Key
!uv pip install --system --upgrade --quiet langchain_community langchain_openai gradio

In [ ]:
# Abschnitt 1: Importe
from IPython.display import display, Markdown

from langchain.output_parsers.structured import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Abschnitt 2: Konstanten definieren
MODEL_NAME = "gpt-4o-mini"
TEMPERATURE = 0.0

# Abschnitt 3: Schema und Parser definieren
response_schemas = [
    ResponseSchema(name="answer", description="Antwort auf die Frage des Benutzers"),
    ResponseSchema(name="source", description="Verwendete Quelle (Website) für die Antwort")
]

parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Abschnitt 4: Prompt-Template erstellen
prompt = PromptTemplate(
    template="""
    Beantworte die folgende Frage so präzise wie möglich.
    Gib auch die Quelle deiner Information an.

    Frage: {question}

    {format_instructions}
    """,
    input_variables=["question"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# Abschnitt 5: LLM-Kette mit LCEL erstellen
chain = (
    {"question": RunnablePassthrough()}
    | prompt
    | ChatOpenAI(model=MODEL_NAME, temperature=TEMPERATURE)
    | parser
)

# Abschnitt 6: Hauptprogramm
def get_answer(question: str) -> dict:
    """ Verarbeitet eine Frage und gibt die strukturierte Antwort zurück. """
    return chain.invoke(question)

# Beispielnutzung
if __name__ == "__main__":
    test_question = "Was ist Machine Learning?"
    result = get_answer(test_question)
    display(Markdown(f"**Antwort:** {result['answer']}"))
    display(Markdown(f"**Quelle:** {result['source']}"))

Hier ist eine Vorlage für eine Eingabeaufforderung (Prompt), die sowohl die gestellte Frage als auch die generierten Formatierungsanweisungen berücksichtigt. Diese kann flexibel an verschiedene Anwendungsfälle angepasst werden.

In [ ]:
question = "Wann wurde Python eingeführt?"

result = get_answer(question)
display(Markdown(f"**Antwort:** {result['answer']}"))
display(Markdown(f"**Quelle:** {result['source']}"))

Wie ersichtlich ist, nutzt LangChain Schemata, um das JSON-Format für die Antwortausgabe festzulegen. Durch dieses Format lassen sich die einzelnen Werte problemlos analysieren.  

Nun wird eine Kette erstellt, die den StructuredOutputParser einbindet und das LLM abfragt.

In [ ]:
MODEL = "gpt-4o-mini"
TEMPERATURE = 0

# Initialisieren des LLM
llm = ChatOpenAI(
    model=MODEL,
    temperature=TEMPERATURE,
    n=1
)
chain = prompt | llm | parser

Jetzt werden eine Frage, die dazugehörige Antwort sowie die Quelle der Frage dargestellt.

In [ ]:
question = "Wann wurde Python eingeführt?"
result = chain.invoke({"question": question})
display(Markdown(f"**Antwort:** {result['answer']}"))
display(Markdown(f"**Quelle:** {result['source']}"))

<p><font color='black' size="5">
Erkennen und Übersetzen
</font></p>

Nun wird ein Beispiel mit einer größeren Anzahl an Werten getestet. Das folgende Programm nimmt Texte in beliebigen Sprachen entgegen und übersetzt sie ins Englische, Spanische und Chinesische.

In [ ]:
response_schemas = [
    ResponseSchema(name="detected", description="Die Sprache ist Nutzer-Eingabe"),
    ResponseSchema(name="english", description="English translation"),
    ResponseSchema(name="spanish", description="Spanish translation"),
    ResponseSchema(name="chinese", description="Chinese translation"),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="Übersetze in die angegebenen Sprachen.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

chain = prompt | llm | output_parser

Zunächst wird ein deutscher Satz getestet, um zu beobachten, wie die Übersetzung in die drei Zielsprachen erfolgt.

In [ ]:
question = "Wann wurde Python eingeführt?"
chain.invoke({"question": question})

# **3 | Datenformat Parser**
---

LangChain bietet eine breite Palette an Parsern, die unterschiedliche Datenformate verarbeiten können, was seine Einsatzmöglichkeiten erheblich erweitert. Es unterstützt unter anderem die nahtlose Integration von Pandas-Datenrahmen, kommagetrennten Listen, JSON-Strukturen sowie Datums- und Zeitobjekten. Diese Flexibilität ermöglicht eine effiziente Anpassung an verschiedene Arten von Dateneingaben und macht LangChain zu einem leistungsstarken Werkzeug für die Analyse und Verarbeitung von Daten. Im Folgenden werden einige dieser Parser genauer betrachtet, ihre praktischen Anwendungen demonstriert und aufgezeigt, wie sie zur Optimierung von Prozessen und zur Gewinnung wertvoller Erkenntnisse beitragen können.

<p><font color='black' size="5">
CSV
</font></p>

Der **CommaSeparatedListOutputParser** ermöglicht die Umwandlung von LLM-Ausgaben in eine durch Kommas getrennte Liste und extrahiert diese als native Python-Liste. Dies ist besonders nützlich, wenn strukturierte Daten aus einem Sprachmodell extrahiert und für weitere Verarbeitungsschritte verwendet werden sollen. Indem der Parser die Ausgabe direkt in eine Listenstruktur überführt, erleichtert er die Handhabung und Weiterverarbeitung der generierten Daten in verschiedenen Anwendungen.

In [ ]:
# Abschnitt 1: Importe
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Abschnitt 3: Konstanten definieren
MODEL = "gpt-4o-mini"
TEMPERATURE = 0

# Abschnitt 4: Chat-Komponenten initialisieren
output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
chat_prompt = PromptTemplate(
    template="List ten {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

chat_model = ChatOpenAI(
    model=MODEL,
    temperature=TEMPERATURE,
    n=1
)

# Abschnitt 5: Funktionen definieren
def generate_list(subject: str):
    """  Generiert eine Liste mit zehn Elementen zu einem gegebenen Thema.  """
    chain = chat_prompt | chat_model | output_parser
    return chain.invoke({"subject": subject})

# Abschnitt 6: Hauptprogramm
def main():
    """Startet die Hauptkonversation und gibt eine Liste von Städten aus."""
    cities = generate_list("Städte")
    print(type(cities), cities)

if __name__ == "__main__":
    main()

<p><font color='black' size="5">
JSON
</font></p>

Die Ausgabe des LLM kann im JSON-Format strukturiert werden. In diesem Beispiel wird ein Satz erkannt, der als Englisch identifiziert wird, und anschließend ins Spanische, Französische und Chinesische übersetzt.

In [ ]:
# Abschnitt 1: Importe
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

# Abschnitt 2: Konstanten und Datenmodell
class Translate(BaseModel):
    """Datenmodell für mehrsprachige Übersetzungen"""
    detected: str = Field(description="Erkannte Ausgangssprache")
    spanish: str = Field(description="Spanische Übersetzung")
    french: str = Field(description="Französische Übersetzung")
    chinese: str = Field(description="Chinesische Übersetzung")

# Abschnitt 3: Funktionen
def translate_text(text: str) -> dict:
    """Übersetzt den Text in mehrere Sprachen"""
    # Parser und LLM konfigurieren
    parser = JsonOutputParser(pydantic_object=Translate)
    llm = ChatOpenAI(model="gpt-4", temperature=0.0)

    # Prompt erstellen und Kette ausführen
    prompt = PromptTemplate(
        template="Übersetze die Benutzereingabe.\n{format_instructions}\n{input}\n",
        input_variables=["input"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )

    chain = prompt | llm | parser
    return chain.invoke({"input": text})

# Abschnitt 4: Hauptprogramm
# Beispieltext übersetzen und Ergebnisse ausgeben
result = translate_text("What is your name?")
print(type(result), result)
print()
print(f"Erkannte Sprache: {result['detected']}")
print(f"Spanisch: {result['spanish']}")
print(f"Französisch: {result['french']}")
print(f"Chinesisch: {result['chinese']}")

<p><font color='black' size="5">
Datum/Uhrzeit
</font></p>

Langchain bietet mit dem **DatetimeOutputParser** eine spezielle Funktion zur Interpretation von Datums- und Zeitangaben aus Texten. Sie erkennt unterschiedliche Formate und wandelt diese in ein standardisiertes Zeitformat um. Dies ist besonders hilfreich für Anwendungen wie Terminplanung, Datenanalyse oder andere Bereiche, in denen eine präzise Verarbeitung zeitlicher Informationen erforderlich ist. Der **DatetimeOutputParser** erleichtert die Verarbeitung von Datums- und Uhrzeitangaben und trägt dazu bei, dass Anwendungen zeitbezogene Daten effizient verwalten und nutzen können.

In [ ]:
# Abschnitt 1: Importe
from langchain.output_parsers import DatetimeOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

# Abschnitt 2: Konstanten
TEMPLATE = """Beantworte die Frage zu genau wie möglich:

{question}

{format_instructions}"""

# Abschnitt 3: Komponenten initialisieren
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.2,
    n=1
)

output_parser = DatetimeOutputParser()
prompt = PromptTemplate(
    template=TEMPLATE,
    input_variables=['question'],
    partial_variables={'format_instructions': output_parser.get_format_instructions()}
)

Wir können die Eingabeaufforderung anzeigen, die wir zum Abrufen von Daten verwenden werden.

In [ ]:
print(prompt)

Wir erstellen die Kette, die wir zum Parsen von Daten verwenden werden.

In [ ]:
chain = prompt | llm | output_parser

Wir werden nach zwei Daten fragen, einem realen und einem fiktiven.

In [ ]:
output = chain.invoke({"question": "Wann wurde die Progammiersprache Python eingeführt?"})
print(output)

In [ ]:
output = chain.invoke({"question": "An welchem Tag startet der Krieg im Videospiel Fallout?"}) #  23. Oktober 2077
print(output)


# **4 | Pydantic Parser**
---

Pydantic ist eine Python-Bibliothek zur Datenvalidierung und Einstellungsverwaltung, die auf den Typanmerkungen von Python basiert. Sie ermöglicht eine effiziente und einfache Validierung sowie Umwandlung von Daten und nutzt dabei das standardisierte Typisierungssystem von Python.

**Wichtige Funktionen von Pydantic:**
- **Datenvalidierung:** Überprüft, ob Daten den erwarteten Formaten entsprechen, und konvertiert sie bei Bedarf in die korrekten Typen.
- **Unterstützung für Code-Editoren:** Dank der Nutzung von Python-Typanmerkungen bieten moderne Editoren Funktionen wie Autovervollständigung und Typprüfung für Pydantic-Modelle.
- **Fehlermeldungen:** Detaillierte und verständliche Fehlermeldungen helfen, Probleme bei der Datenvalidierung gezielt zu identifizieren.
- **Einstellungsverwaltung:** Erleichtert das Laden und Verwalten von Konfigurationsparametern aus verschiedenen Quellen, darunter Umgebungsvariablen und JSON-Dateien.
- **Erweiterbarkeit:** Modelle lassen sich durch Methoden und Eigenschaften anpassen, und benutzerdefinierte Validierungen können mithilfe von Pydantic-Dekoratoren hinzugefügt werden.
- **Integration mit anderen Bibliotheken:** Besonders in Verbindung mit FastAPI verbessert Pydantic die Verwaltung und Validierung von API-Daten erheblich.

Aufgrund seiner Flexibilität und Benutzerfreundlichkeit ist Pydantic besonders nützlich für Webentwicklung und datenintensive Anwendungen, bei denen eine zuverlässige Datenverarbeitung erforderlich ist.



**Pydantic in LangChain:**    
LangChain, eine Bibliothek zur Entwicklung von Anwendungen mit Sprachmodellen, nutzt Pydantic für die strukturierte Verarbeitung von Modellantworten. Der **PydanticOutputParser** gewährleistet, dass die Ausgaben eines Sprachmodells einer vorgegebenen Struktur entsprechen. Dies ist besonders vorteilhaft für Anwendungen, bei denen konsistente Datenformate entscheidend sind – beispielsweise bei der Datenextraktion, API-Antworten oder der automatisierten Weiterverarbeitung von Modell-Ausgaben.

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field, validator
from langchain_openai import ChatOpenAI

MODEL = "gpt-4o-mini"
TEMPERATURE = 0

# Initialisieren des OpenAI LLM
llm = ChatOpenAI(
    model=MODEL,
    temperature=TEMPERATURE,
    n=1
)

Hier ist ein Beispielcode, der ein Sprachmodell (LLM) verwendet, um einen Witz zu generieren, wobei der PydanticOutputParser sicherstellt, dass die Ausgabe mit einem Fragezeichen endet. Diese Validierung hilft dabei, eine konsistente Struktur zu gewährleisten, insbesondere da LLMs manchmal unvorhersehbare Antworten liefern können.

**Erklärung:**   
Das Programm ist ein einfacher Filmografie-Generator mit einer Geschlechtervalidierung. Hier die Hauptfunktionen:

1. Es definiert eine Klasse `Schauspieler` mit drei Feldern:
   - Name des Schauspielers
   - Geschlecht (m/w)
   - Liste der Filme

2. Es enthält einen Validator, der prüft ob das Geschlecht 'männlich' (m) ist. Falls ein weibliches Geschlecht (w) eingegeben wird, erzeugt es einen Fehler.

3. Das Programm nutzt ChatGPT (über die OpenAI API) um:
   - Den Namen zu erkennen
   - Das Geschlecht zu bestimmen
   - Eine Liste von Filmen zu generieren

4. Wenn man nach einer Schauspielerin fragt (wie Emma Stone), gibt das Programm einen Fehler aus, da der Validator nur männliche Schauspieler zulässt.

Im konkreten Beispiel wird nach Emma Stone gefragt - das Programm erkennt sie als Schauspielerin (w), aber der Validator löst einen Fehler aus, da nur männliche Schauspieler (m) erlaubt sind.

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field, field_validator
from langchain.schema.runnable import RunnablePassthrough

class Schauspieler(BaseModel):
    name: str = Field(description="Name des Schauspielers/der Schauspielerin")
    geschlecht: str = Field(description="Geschlecht (m/w)")
    filmnamen: list[str] = Field(description="Liste der Filme")

    @field_validator("geschlecht")
    def validiere_geschlecht(cls, v: str) -> str:
        if v.lower() != 'm': raise ValueError("Geschlecht muss 'männlich' sein")
        return v.lower()

# Chain Komponenten erstellen
parser = PydanticOutputParser(pydantic_object=Schauspieler)
llm = ChatOpenAI(temperature=0)

# Prompt Template
prompt = PromptTemplate(
    template="Beantworte die Nutzeranfrage:\n{formatierungsanweisungen}\n{abfrage}",
    input_variables=["abfrage"],
    partial_variables={"formatierungsanweisungen": parser.get_format_instructions()},
)

# LCEL Chain mit Pipe-Operator
chain = ({"abfrage": RunnablePassthrough()} | prompt | llm | parser)

# Ausführung
try:
    frage = "Erstelle eine Filmografie für Emma Stone."
    ergebnis = chain.invoke(frage)
    print(f"Name: {ergebnis.name} ({ergebnis.geschlecht})")
    print(f"Filme: {', '.join(ergebnis.filmnamen)}")
except ValueError as e:
    print(f"Fehler: {e}")

Kurze Fehlermeldung:

In [ ]:
try:
    frage = "Erstelle eine Filmografie für Emma Stone."
    ergebnis = (prompt | llm | parser).invoke({"abfrage": frage})
    print(f"Name: {ergebnis.name} ({ergebnis.geschlecht})")
    print(f"Filme: {', '.join(ergebnis.filmnamen)}")
except ValueError as e:
    # Nur die wichtigste Fehlermeldung ausgeben
    error_msg = str(e)
    if "Value error" in error_msg:
        print(f"Fehler: {error_msg.split('Value error,')[1].split('[')[0].strip()}")
    else:
        print(f"Fehler: {error_msg}")

# **5 | Eigene Parser**
---

In bestimmten Szenarien kann es sinnvoll sein, einen benutzerdefinierten Parser zu erstellen, um die Modellausgabe eindeutig zu formatieren.  

Dafür bietet sich die Verwendung von **RunnableLambda** oder **RunnableGenerator** in LCEL an, was für viele Fälle ein guten Ansatz ist.  

<p><font color='black' size="5">
Trennen Erläuterung/Code
</font></p>



Large Language Models (LLMs) wie GPT-4 können Text generieren, der Code und erklärende Beschreibungen nahtlos vermischt. Dies kann zwar für Lern- und Dokumentationszwecke unglaublich nützlich sein, kann aber eine Herausforderung darstellen, wenn aus solchen Ausgaben mit gemischtem Inhalt nur der Code extrahiert und ausgeführt werden muss. Um dies zu beheben, implementieren wir eine einfache Funktion, die nicht-Python-Codezeilen aus einer gegebenen Textzeichenfolge entfernt.

Bei diesem Ansatz werden reguläre Ausdrücke verwendet, um Zeilen zu identifizieren und beizubehalten, die der typischen Python-Syntax entsprechen, während Zeilen verworfen werden, die beschreibender Text zu sein scheinen. Aufgrund der inhärenten Komplexität und Variabilität sowohl von Python-Code als auch von natürlicher Sprache kann diese Methode jedoch nie perfekt sein. Sie basiert auf heuristischen Mustern, die Code manchmal fälschlicherweise als Text klassifizieren oder umgekehrt.

Im nächsten Abschnitt werden wir untersuchen, wie ein anderes LLM beim Entfernen von Nicht-Python-Code helfen kann und möglicherweise eine ausgefeiltere und genauere Lösung bietet. Das folgende Beispiel enthält eine Mischung aus LLM-Kommentaren und generiertem Code.









In [ ]:
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown


# 1. LLM-Modell definieren
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 2. Funktion zur Trennung von Erläuterung und Code
def parse_ai_message(ai_message):
    """Trennt die Erläuterung und den Code aus einer AIMessage und gibt sie separat zurück."""
    text = ai_message.content  # Extrahiere den reinen Textinhalt der AIMessage

    if "```" in text:
        # Trennen der Erläuterung und des Codes
        parts = text.split("```")
        explanation = parts[0].strip()
        code = parts[1].strip() if len(parts) > 1 else ""
    else:
        # Falls kein Codeblock vorhanden ist, geben wir nur die Erläuterung zurück
        explanation = text.strip()
        code = ""

    return explanation, code  # Rückgabe von zwei separaten Werten

# 3. RunnableLambda für das Parsing erstellen
parser = RunnableLambda(parse_ai_message)

# 4. LLM und Parser verketten (Pipeline)
pipeline = llm | parser

# 5. Eingabe an die Pipeline senden und Ergebnis ausgeben
explanation, code = pipeline.invoke("Erkläre mir, wie man eine einfache Funktion in Python erstellt, und gib ein Beispiel.")

display(Markdown("## Erläuterung"))
display(Markdown(explanation))
display(Markdown("## Code"))
print(code)

# **A | Aufgabe**
---

Die Aufgabestellungen unten bieten Anregungen, Sie können aber auch gerne eine andere Herausforderung angehen.


<p><font color='black' size="5">
JSON-Parser mit LangChain
</font></p>


**Ziel:** Verständnis für den Einsatz von `JsonOutputParser` in LangChain.

**Aufgabe:**  
1. Nutze den `JsonOutputParser` von LangChain, um eine KI-Antwort in JSON zu formatieren.  
2. Lasse ein Language Model (z. B. OpenAI GPT) eine Liste von drei zufälligen Städten in Deutschland generieren.  
3. Verwende den Parser, um die Ausgabe in ein JSON-Format umzuwandeln.

**Erwartete Ausgabe (Beispiel):**
```json
{
  "cities": ["Berlin", "Hamburg", "München"]
}
```



<p><font color='black' size="5">
Extraktion von Schlüsselwerten
</font></p>


**Ziel:** Nutzung des `PydanticOutputParser`, um strukturierte Daten aus natürlicher Sprache zu extrahieren.

**Aufgabe:**  
1. Definiere eine `Pydantic`-Datenklasse mit den Feldern: `name` (str), `alter` (int), `stadt` (str).  
2. Verwende ein Language Model, um aus einer gegebenen Beschreibung eine Person zu extrahieren.  
3. Parse die Antwort mit dem `PydanticOutputParser`.

**Beispiel-Eingabe:**
> "Max Mustermann ist 35 Jahre alt und lebt in Berlin."

**Erwartete Ausgabe (Pydantic-Modell):**
```json
{
  "name": "Max Mustermann",
  "alter": 35,
  "stadt": "Berlin"
}
```


<p><font color='black' size="5">
Parser für Listenformate
</font></p>


**Ziel:** Implementierung eines eigenen Parsers zur Umwandlung von KI-Ausgaben in Listen.

**Aufgabe:**  
1. Erstelle eine eigene Parser-Klasse, die eine durch Kommas getrennte Liste in ein Listenformat umwandelt.  
2. Verwende diesen Parser, um eine Liste von fünf beliebten Büchern aus einer Language-Model-Antwort zu extrahieren.

**Beispiel-Eingabe:**
> "Die Verwandlung, Faust, Der Prozess, Die Blechtrommel, Der Vorleser"

**Erwartete Ausgabe:**
```python
["Die Verwandlung", "Faust", "Der Prozess", "Die Blechtrommel", "Der Vorleser"]
```

<p><font color='black' size="5">
Kombination Parser & PromptTemplate
</font></p>


**Ziel:** Verwendung von `StructuredOutputParser` in Kombination mit `PromptTemplate`.

**Aufgabe:**  
1. Erstelle ein `PromptTemplate`, das eine strukturierte Antwort über ein Land ausgibt (Name, Hauptstadt, Einwohnerzahl).  
2. Verwende den `StructuredOutputParser`, um die Antwort in ein Dictionary zu konvertieren.

**Beispiel-Prompt:**
> "Gib mir Informationen zu Frankreich im folgenden JSON-Format: { 'name': '...', 'hauptstadt': '...', 'einwohner': ... }."

**Erwartete Ausgabe:**
```json
{
  "name": "Frankreich",
  "hauptstadt": "Paris",
  "einwohner": 67000000
}
```
